# Подготовка датасета для соревнования [SF-DST] Car Price prediction
В данном ноутбуке мы преобразуем ранее выгруженные с авто.ру json-файлы в датасет, который подгрузим на kaggle

In [1]:
import json
import pandas as pd

Откроем 2 ранее выгруженных json-файла и сделаем из них один.

In [2]:
with open("data.json", "r", encoding="utf8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

with open("data2.json", "r", encoding="utf8") as f:
    data = json.load(f)
df2 = pd.DataFrame(data)

df = df.append(df2, sort=True, ignore_index=True)

Посмотрим на получившийся датасет. Также нам поможет JSON Viewer: http://jsonviewer.stack.hu/

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6753 entries, 0 to 6752
Data columns (total 39 columns):
additional_info      6753 non-null object
autostrategies       6753 non-null object
availability         6753 non-null object
badges               41 non-null object
brand_cert_info      6753 non-null object
category             6753 non-null object
color_hex            6753 non-null object
counters             6753 non-null object
credit_info          6753 non-null object
daily_counters       6753 non-null object
delivery_info        6753 non-null object
description          6563 non-null object
discount_options     147 non-null object
discount_price       35 non-null object
documents            6753 non-null object
grouping_info        0 non-null object
hash                 6753 non-null object
id                   6753 non-null object
is_favorite          6753 non-null bool
lk_summary           6753 non-null object
old_category_id      6753 non-null int64
owner_expenses       6

,additional_info,autostrategies,availability,badges,brand_cert_info,category,color_hex,counters,credit_info,daily_counters,...,seller_type,service_prices,service_schedules,services,soldMinutes,state,status,sub_category,tags,vehicle_info
0,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 195000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, oversize, medium, fivestars, pres...","{'configuration': {'id': '4601199', 'body_type..."
1,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 219000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, big, medium, fivestars, prestige,...","{'configuration': {'id': '6088623', 'body_type..."
2,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,"[Обслуживалась у дилера, Оригинальный пробег, ...",{},cars,97948F,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 155000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, big, oversize, discount, medium, ...","{'configuration': {'id': '2305680', 'body_type..."
3,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,200204,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},[],NaN,"{'mileage': 348000, 'state_not_beaten': True, ...",ACTIVE,cars,"[oversize, medium, fivestars, prestige, comfor...","{'configuration': {'id': '4601199', 'body_type..."
4,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,"[Один владелец, Максимальная комплектация, В и...",{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 150000, 'state_not_beaten': True, ...",ACTIVE,cars,"[proven_owner, big, prestige, comfort, badge, ...","{'configuration': {'id': '3619826', 'body_type..."


Посмотрим, какие данные мы сможем забрать. Начнем по порядку.

## Additional Info

In [4]:
df.additional_info[0]

{'hidden': False,
 'is_on_moderation': False,
 'not_disturb': True,
 'update_date': '1594724311000',
 'creation_date': '1594289299898',
 'mobile_autoservices_url': 'https://m.auto.ru/autoservice/all_works/BMW/?geo_id=213',
 'hot_info': {'is_hot': False,
  'start_time': '2020-07-09T10:05:52Z',
  'end_time': '2020-07-09T10:05:52Z'},
 'fresh_date': '1594724292980',
 'counters_start_date': '1594289299898',
 'chat_only': False,
 'dealer_card_promo': 'DONT_SHOW',
 'review_summary': {'avg_rating': 4.599999904632568, 'counter': 62},
 'price_stats': {'last_year_price_percentage_diff': -14},
 'booking': {}}

Здесь может пригодиться рейтинг авто. Сделаем такой признак.

In [5]:
df['rating'] = df['additional_info'].apply(lambda x: x.get('review_summary').get('avg_rating'))

In [6]:
df = df.drop('additional_info', axis=1)

## Autostrategies
Тут пусто, удалим

In [7]:
df = df.drop('autostrategies', axis=1)

## Availability

In [8]:
df.availability.value_counts()

IN_STOCK    6751
ON_ORDER       2
Name: availability, dtype: int64

Видимо, некоторые салоны привозят авто под заказ. Не думаю, что это сильно повлияет на цену. Удалим этот признак.

In [9]:
df = df.drop('availability', axis=1)

## Badges, brand_cert_info, category
Ничего интересного, удалаяем

In [10]:
df = df.drop('badges', axis=1)
df = df.drop('brand_cert_info', axis=1)
df = df.drop('category', axis=1)

## Color_hex
Цвет машины, записан в виде кодировки RGB. Переделаем в название цветов на английском.

In [11]:
df.color_hex.value_counts()

040001    2733
FAFBFB    1134
0000CC     845
97948F     788
CACECB     424
200204     257
EE1D19     200
007F00     106
22A0F8      72
C49648      61
660099      40
DEA522      38
FF8649      29
4A2197      20
FFD600       5
FFC0CB       1
Name: color_hex, dtype: int64

In [12]:
dic_color = {
    '040001': 'black',
    'FAFBFB': 'white',
    '97948F': 'grey',
    '0000CC': 'dark_blue',
    'CACECB': 'silver', 
    'EE1D19': 'red',
    '200204': 'brown',
    '007F00': 'green',
    'C49648': 'biege',
    '22A0F8': 'blue',
    '660099': 'purple',
    'FF8649': 'orange',
    'DEA522': 'golden',
    '4A2197': 'violet',
    'FFD600': 'yellow', 
}
df['color_hex'] = df['color_hex'].map(dic_color)
df.rename(columns = {'color_hex':'color'}, inplace = True) 

## Counters, credit_info, daily_counters, delivery_info 
Удаляем

In [13]:
df = df.drop('counters', axis=1)
df = df.drop('credit_info', axis=1)
df = df.drop('daily_counters', axis=1)
df = df.drop('delivery_info', axis=1)

## Description

In [14]:
df.description[0]

'Автомобиль в отличном состоянии. Ни каких нареканий по лакокрасочному покрытию. Салон чистый и ухоженный, не прокурен. Двигатель работает в штатном режиме, не дымит и не троит. Коробка переключает плавно и без рывков. Отсутствуют посторонние шумы в ходовой части. Машина прошла предпродажную диагностику ДВС и АКПП, в результате которой, не выявлено никаких  отклонений. \n-А также вы можете приобрести данный автомобиль в кредит!!!\n-Рассмотрим вариант обмена!!!\n-Гарантируем юридическую чистоту автомобиля!!!'

Интересно было бы проанализировать, но вряд ли успеем в срок. Удаляем.

In [15]:
df = df.drop('description', axis=1)

## Discount_options, discount_price
Не очень интересно и много пропусков. Удаляем.

In [16]:
df = df.drop('discount_options', axis=1)
df = df.drop('discount_price', axis=1)

## Documents

In [17]:
df.documents[0]

{'owners_number': 3,
 'pts_original': True,
 'custom_cleared': True,
 'year': 2004,
 'vin': 'WBA**************',
 'pts': 'ORIGINAL',
 'license_plate': '******|152',
 'vin_resolution': 'OK'}

Отсюда мы можем взять количество владельцев, ПТС оригинал или дубликат и год выпуска.

### Количество владельцев - owners

Забираем данные о количестве владельцев:

In [18]:
df['owners'] = df['documents'].apply(lambda x: x.get('owners_number'))

In [19]:
df.owners.value_counts()

3.0    3313
2.0    1758
1.0    1532
Name: owners, dtype: int64

In [20]:
print('Количество пропусков: {},'.format(df.owners.isnull().sum()))

Количество пропусков: 150,


Видим, что довольно много пропусков. Пока что оставим как есть.

### Данные о ПТС - technicalPassport

Забираем данные о ПТС:

In [21]:
df['technicalPassport'] = df['documents'].apply(lambda x: x.get('pts'))

In [22]:
df.technicalPassport.value_counts()

ORIGINAL     5741
DUPLICATE    1010
Name: technicalPassport, dtype: int64

In [23]:
print('Количество пропусков: {},'.format(df.technicalPassport.isnull().sum()))

Количество пропусков: 2,


Отлично, всего 2 пропуска. Приведем значения к виду как на kaggle.

In [24]:
dic_technicalPassport = {'ORIGINAL': 1, 'DUPLICATE': 0}
df['technicalPassport'] = df['technicalPassport'].map(dic_technicalPassport)

### Возраст машины - age

Забираем данные о годе выпуска машины по документам:

In [25]:
df['age'] = df['documents'].apply(lambda x: x.get('year'))

In [26]:
df.age.value_counts()

2013    599
2011    566
2012    561
2008    486
2019    444
2010    381
2018    366
2014    345
2017    341
2009    325
2016    309
2007    308
2015    262
2004    175
2005    158
2006    149
2003    136
2001    126
2002    122
1998     90
1999     81
1997     79
2000     75
1996     37
1991     31
1993     31
1990     27
1992     25
1994     23
1995     20
1988     14
1989     12
1987      8
1984      6
1985      5
1986      4
1937      4
1980      3
1983      3
1982      3
1950      2
1972      1
1968      1
1953      1
1975      1
1949      1
1978      1
1969      1
1981      1
1947      1
1938      1
1948      1
Name: age, dtype: int64

In [27]:
df['age']= 2020-df['age']

Здесь мы все что нужно забрали. Удалим столбец.

In [28]:
df = df.drop('documents', axis=1)

## grouping_info, hash, id, is_favorite, lk_summary, lk_summary, old_category_id, owner_expenses, price_history

Удаляем т.к.:
* grouping_info - пусто
* hash - не интересно
* id - не интересно
* is_favorite - не интересно
* lk_summary - содержит информацию, которая есть в других столбцах
* old_category_id - не интересно
* owner_expenses - данные о налогах и страховках. Налоги коррелируют с мощностью, страховка не интересно
* price_history - нас интересует только последняя цена

In [29]:
for columns in [
    'grouping_info',
    'hash',
    'id',
    'is_favorite',
    'lk_summary',
    'old_category_id',
    'owner_expenses',
    'price_history',
]:
    df = df.drop(columns, axis=1)

## Price_info
Цена в разных валютах

Добавим информацию в наш датафрейм.

In [30]:
df['price_rur'] = df['price_info'].apply(lambda x: x.get('RUR'))
df['price_usd'] = df['price_info'].apply(lambda x: x.get('USD'))
df['price_eur'] = df['price_info'].apply(lambda x: x.get('EUR'))

In [31]:
df['price_rur'].mean()/df['price_usd'].mean()

71.1275414960321

Похоже на сегодняшний курс. Логично, что авто.ру сам переводит рублевые цены в доллары.
Удалим цены в зарубежных валютах и price_info.

In [32]:
for columns in [
    'price_info',
    'price_usd',
    'price_eur',
]:
    df = df.drop(columns, axis=1)

### recall_info, saleId, salon, section, seller, seller_type, service_prices, service_schedules, services, soldMinutes, status, sub_category
Либо пусто, либо ничего полезного

In [33]:
for columns in [
    'recall_info',
    'saleId',
    'salon',
    'section',
    'seller',
    'seller_type',
    'service_prices',
    'service_schedules',
    'services',
    'soldMinutes',
    'status',
    'sub_category',
]:
    df = df.drop(columns, axis=1)

## State
Состояние. Отсюда мы можем взять пробег.

In [34]:
df['mileage'] = df['state'].apply(lambda x: x.get('mileage'))

Здесь ещё есть информация о том, является ли автомобиль новым или с пробегом. Но в тестовой выборке все авто с пробегом.

In [35]:
df = df.drop('state', axis=1)

## tags
Тэги. Думаю, что тэги будут неплохими дополнительными параметрами для нашей модели.

In [36]:
df.tags[0]

['good_price',
 'oversize',
 'medium',
 'fivestars',
 'prestige',
 'comfort',
 'allowed_for_credit',
 'autoru_exclusive',
 'wide-back-seats',
 'big-trunk',
 'all-terrain',
 'vin_offers_history',
 'options',
 'available_for_checkup',
 'handling',
 'vin_resolution_ok',
 'vin_checked',
 'pts_original',
 'equitable_price',
 'high_reviews_mark']

Вот только непонятно, как мы обогатим этими парметрами данные из тестовой выборки. Удалим столбец, но в будущем подумаем о возможных тэгах.

In [37]:
df = df.drop('tags', axis=1)

## vehicle_info
Информация об автомобиле

In [38]:
df.vehicle_info[25]

{'configuration': {'id': '2305608',
  'body_type': 'SEDAN',
  'doors_count': 4,
  'auto_class': 'E',
  'human_name': 'Седан',
  'trunk_volume_min': 520,
  'body_type_group': 'SEDAN',
  'main_photo': {'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/orig',
    'wizardv3mr': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/wizardv3mr',
    'cattouch': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/cattouch'}},
  'tags': ['big', 'big-trunk', 'wide-back-seats', 'medium']},
 'complectation': {'id': '4720222',
  'name': '535',
  'available_options': ['cruise-control',
   'airbag-passenger',
   'lock',
   'electro-mirrors',
   'mirrors-heat',
   'leather',
   'computer',
   'wheel-power',
   'airbag-side',
   'abs',
   'climate-control-1',
   'esp',
   'audiopreparation',
   'electro-window-back',
   'condition',
   'airbag-driver',
   'electro-window-front',
   'ptf',
   'audiosys

Пойдем по порядку

### body_Type
Тип кузова. Приведем в вид, аналогичный тестовому датасету на Kaggle.

In [39]:
df['bodyType'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('body_type'))

In [40]:
df.bodyType.value_counts()

SEDAN                3053
ALLROAD_5_DOORS      2704
COUPE                 341
HATCHBACK_5_DOORS     240
LIFTBACK              159
WAGON_5_DOORS          83
HATCHBACK_3_DOORS      72
CABRIO                 58
ROADSTER               27
SEDAN_2_DOORS           9
COMPACTVAN              5
COUPE_HARDTOP           2
Name: bodyType, dtype: int64

In [41]:
dic_bodyType = {
    'SEDAN': 'sedan',
    'SEDAN_2_DOORS': 'sedan',
    'ALLROAD_5_DOORS': 'SUV',
    'HATCHBACK_5_DOORS': 'hatchback',
    'HATCHBACK_3_DOORS': 'hatchback',
    'COUPE': 'coupe',
    'COUPE_HARDTOP': 'coupe',    
    'ROADSTER': 'roadster',
    'LIFTBACK': 'liftback',
    'WAGON_5_DOORS': 'wagon',
    'CABRIO': 'cabriolet',
    'COMPACTVAN': 'van'
}
df['bodyType'] = df['bodyType'].map(dic_bodyType)

### doors_count
Количество дверей

In [42]:
df['numberOfDoors'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('doors_count'))

In [43]:
df.numberOfDoors.value_counts()

5    3191
4    3053
2     437
3      72
Name: numberOfDoors, dtype: int64

### Auto_class
Класс автомобиля. Добавим этот признак.

In [44]:
df['autoClass'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('auto_class'))

### Complectation
Комплектация

In [45]:
df.vehicle_info[25].get('complectation')

{'id': '4720222',
 'name': '535',
 'available_options': ['cruise-control',
  'airbag-passenger',
  'lock',
  'electro-mirrors',
  'mirrors-heat',
  'leather',
  'computer',
  'wheel-power',
  'airbag-side',
  'abs',
  'climate-control-1',
  'esp',
  'audiopreparation',
  'electro-window-back',
  'condition',
  'airbag-driver',
  'electro-window-front',
  'ptf',
  'audiosystem-cd',
  'wheel-configuration2',
  'wheel-configuration1',
  'immo']}

Заберем список опций (available_options) в виде списка. Далее сделаем dummy-переменные.

In [46]:
df['complectation'] = df['vehicle_info'].apply(lambda x: x.get('complectation').get('available_options'))

In [47]:
df['complectation'][25]

['cruise-control',
 'airbag-passenger',
 'lock',
 'electro-mirrors',
 'mirrors-heat',
 'leather',
 'computer',
 'wheel-power',
 'airbag-side',
 'abs',
 'climate-control-1',
 'esp',
 'audiopreparation',
 'electro-window-back',
 'condition',
 'airbag-driver',
 'electro-window-front',
 'ptf',
 'audiosystem-cd',
 'wheel-configuration2',
 'wheel-configuration1',
 'immo']

Проверим количество пропусков

In [48]:
print('Количество пропусков: {},'.format(df['complectation'].isnull().sum()))

Количество пропусков: 4626,


Очень много пропусков. Удалим этот признак т.к. в признаке 'equipment' также будут перечислены различные опции.

In [49]:
df = df.drop('complectation', axis=1)

### Equipment
Оборудование

In [50]:
df.vehicle_info[25].get('equipment')

{'cruise-control': True,
 'asr': True,
 'tinted-glass': True,
 'esp': True,
 'adaptive-light': True,
 'usb': True,
 'sport-seats': True,
 'e-adjustment-wheel': True,
 'multi-wheel': True,
 'xenon': True,
 'audiosystem-tv': True,
 'ashtray-and-cigarette-lighter': True,
 'airbag-passenger': True,
 'front-centre-armrest': True,
 'navigation': True,
 'front-seat-support': True,
 'decorative-interior-lighting': True,
 'isofix-front': True,
 'lock': True,
 'door-sill-panel': True,
 'electro-mirrors': True,
 'drl': True,
 'electro-window-back': True,
 'reduce-spare-wheel': True,
 'multizone-climate-control': True,
 'mirrors-heat': True,
 'power-latching-doors': True,
 'hatch': True,
 'volume-sensor': True,
 'dark-interior': True,
 'sport-suspension': True,
 'folding-front-passenger-seat': True,
 'leather': True,
 '19-inch-wheels': True,
 'driver-seat-memory': True,
 'sport-pedals': True,
 'engine-proof': True,
 'front-seats-heat-vent': True,
 'start-stop-function': True,
 'airbag-driver': Tru

Преобразуем словарь в список ключей и запишем в переменную equioment.

In [51]:
df['equipment'] = df['vehicle_info'].apply(lambda x: list(x.get('equipment').keys()))

In [52]:
df.equipment[25]

['cruise-control',
 'asr',
 'tinted-glass',
 'esp',
 'adaptive-light',
 'usb',
 'sport-seats',
 'e-adjustment-wheel',
 'multi-wheel',
 'xenon',
 'audiosystem-tv',
 'ashtray-and-cigarette-lighter',
 'airbag-passenger',
 'front-centre-armrest',
 'navigation',
 'front-seat-support',
 'decorative-interior-lighting',
 'isofix-front',
 'lock',
 'door-sill-panel',
 'electro-mirrors',
 'drl',
 'electro-window-back',
 'reduce-spare-wheel',
 'multizone-climate-control',
 'mirrors-heat',
 'power-latching-doors',
 'hatch',
 'volume-sensor',
 'dark-interior',
 'sport-suspension',
 'folding-front-passenger-seat',
 'leather',
 '19-inch-wheels',
 'driver-seat-memory',
 'sport-pedals',
 'engine-proof',
 'front-seats-heat-vent',
 'start-stop-function',
 'airbag-driver',
 'aux',
 'isofix',
 'electro-window-front',
 'projection-display',
 'light-sensor',
 'automatic-lighting-control',
 'passenger-seat-updown',
 'computer',
 'high-beam-assist',
 'keyless-entry',
 'seat-transformation',
 'wheel-power',
 'al

### Tags
Эти тэги нам уже больше подойдут т.к. относятся к модели машины.

In [53]:
df['tags'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('tags'))

В дальнейшем сделаем их тэгов dummy-переменные

### model_info
Название модели

In [54]:
df['model'] = df['vehicle_info'].apply(lambda x: x.get('model_info').get('code'))

In [55]:
df.model.value_counts()

5ER              1658
3ER              1145
X5               1052
7ER               553
X6                521
X3                446
X1                319
1ER               309
6ER               141
X4                106
X6_M               97
X7                 80
4                  78
X5_M               61
M5                 39
Z4                 22
M3                 19
X2                 18
M6                 15
2ER                10
M4                  9
8ER                 8
321                 6
I3                  6
I8                  5
Z3                  4
2GRANDTOURER        4
M2                  4
340                 3
X4_M                2
X3_M                2
M8                  2
326                 2
E3                  1
02                  1
2ACTIVETOURER       1
Z4_M                1
Z1                  1
2000_C_CS           1
Z8                  1
Name: model, dtype: int64

Возможно, в дальнейшем сократим количество моделей путем объединения или удаления.

### super_gen
Поколение. Думаю, что будет сильно коррелировать с годом производства. Думаю, что от поколения цена сильно зависит. Заберем отсюда год как в тестовой выборке.

In [ ]:
df['model'] = df['vehicle_info'].apply(lambda x: x.get('super_gen').get('code'))

### tech_param
Здесь много полезной информации

In [56]:
df.vehicle_info[25].get('tech_param')

{'id': '2305626',
 'name': '535',
 'nameplate': '535d',
 'displacement': 2993,
 'engine_type': 'DIESEL',
 'gear_type': 'REAR_DRIVE',
 'transmission': 'AUTOMATIC',
 'power': 286,
 'power_kvt': 210,
 'human_name': '535d 3.0d AT (286 л.с.)',
 'acceleration': 6.4,
 'clearance_min': 142,
 'fuel_rate': 6.7}

#### name

In [57]:
df['name'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('name'))

#### dispacement
Объем двигателя

In [58]:
df['engineDisplacement'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('displacement'))

In [59]:
df['engineDisplacement'].value_counts()

2993    1234
1995    1232
2979     748
1997     686
4395     475
        ... 
2495       1
2693       1
2479       1
2443       1
1665       1
Name: engineDisplacement, Length: 62, dtype: int64

Довольно много значений, скорее всего будем разделять на категории по литрам с точностью до 0.5 (2,2.5,3л.)

#### engine_type
Тип топлива

In [60]:
df['fuelType'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('engine_type'))

Приведем значения к уже имеющимся на kaggle.

In [61]:
df['fuelType'].value_counts()

GASOLINE    4621
DIESEL      2113
HYBRID        15
ELECTRO        4
Name: fuelType, dtype: int64

In [62]:
dic_fuelType = {'DIESEL': 'diesel', 'GASOLINE': 'petrol', 'HYBRID': 'hybrid', 'ELECTRO': 'electro'}
df['fuelType'] = df['fuelType'].map(dic_fuelType)

#### gear_type
Привод

In [63]:
df['drive'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('gear_type'))

In [64]:
df.drive.value_counts()

ALL_WHEEL_DRIVE    3881
REAR_DRIVE         2851
FORWARD_CONTROL      21
Name: drive, dtype: int64

Приведем значения к уже имеющимся на kaggle.

In [65]:
dic_drive = {'ALL_WHEEL_DRIVE': '4wd', 'REAR_DRIVE': 'rwd', 'FORWARD_CONTROL': 'fwd'}
df['drive'] = df['drive'].map(dic_drive)

#### transmission
Тип коробки передач

In [66]:
df['vehicleTransmission'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('transmission'))

In [67]:
df.vehicleTransmission.value_counts()

AUTOMATIC     6196
MECHANICAL     481
ROBOT           76
Name: vehicleTransmission, dtype: int64

Приведем значения к уже имеющимся на kaggle.

In [68]:
dic_vehicleTransmission = {'AUTOMATIC': 'AT', 'MECHANICAL': 'MT', 'ROBOT': 'AM'}
df['vehicleTransmission'] = df['vehicleTransmission'].map(dic_vehicleTransmission)

#### power
Мощность двигателя

In [69]:
df['enginePower'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('power'))

In [70]:
df['enginePower'].value_counts()

184    818
306    459
249    443
218    434
245    415
      ... 
610      1
480      1
230      1
109      1
449      1
Name: enginePower, Length: 83, dtype: int64

Тут видимо будем тоже на категории делить.

#### acceleration
Разгон до 100 км/ч

In [71]:
df['acceleration'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('acceleration'))

In [72]:
df['acceleration'].value_counts()

8.1     405
7.6     353
6.7     317
7.5     312
6.5     256
       ... 
14.5      1
13.0      1
12.8      1
11.1      1
11.4      1
Name: acceleration, Length: 98, dtype: int64

Здесь можно будет сократить количество параметров, например, округлив до целого.

#### fuel_rate
Расход топлива

In [73]:
df['fuel_rate'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('fuel_rate'))

In [74]:
df['fuel_rate'].value_counts()

6.2     319
10.1    250
6.0     227
5.6     203
10.3    179
       ... 
12.3      2
11.6      2
11.5      2
13.4      1
3.3       1
Name: fuel_rate, Length: 101, dtype: int64

Здесь можно будет сократить количество параметров, например, округлив до целого.

Удалим 'vehicle_info' т.к. все необходимое забрали. Возможно стоило что-то ещё вытянуть, подумаем над этим, если останется время.

In [75]:
df = df.drop('vehicle_info', axis=1)

# Итоги
Посмотрим на получившийся датасет

In [76]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6753 entries, 0 to 6752
Data columns (total 21 columns):
color                  6752 non-null object
rating                 6491 non-null float64
owners                 6603 non-null float64
technicalPassport      6751 non-null float64
age                    6753 non-null int64
price_rur              6737 non-null float64
mileage                6753 non-null int64
bodyType               6753 non-null object
numberOfDoors          6753 non-null int64
autoClass              6740 non-null object
equipment              6753 non-null object
tags                   6749 non-null object
model                  6753 non-null object
name                   6475 non-null object
engineDisplacement     6753 non-null int64
fuelType               6753 non-null object
drive                  6753 non-null object
vehicleTransmission    6753 non-null object
enginePower            6753 non-null int64
acceleration           6717 non-null float64
fuel_rate    

,color,rating,owners,technicalPassport,age,price_rur,mileage,bodyType,numberOfDoors,autoClass,...,tags,model,name,engineDisplacement,fuelType,drive,vehicleTransmission,enginePower,acceleration,fuel_rate
0,black,4.6,3.0,1.0,16,665000.0,195000,SUV,5,J,...,"[big-trunk, oversize, wide-back-seats, all-ter...",X5,3.0d,2993,diesel,4wd,AT,218,8.8,NaN
1,black,4.8,3.0,1.0,9,899000.0,219000,sedan,4,E,...,"[big, big-trunk, wide-back-seats, medium, five...",5ER,520,1995,diesel,rwd,AT,184,8.1,5.2
2,grey,4.6,3.0,1.0,13,790000.0,155000,SUV,5,J,...,"[big, big-trunk, oversize, wide-back-seats, al...",X5,48i,4799,petrol,4wd,AT,355,6.5,12.0
3,brown,4.6,3.0,1.0,16,530000.0,348000,SUV,5,J,...,"[big-trunk, oversize, wide-back-seats, all-ter...",X5,3.0d,2993,diesel,4wd,AT,218,8.8,NaN
4,black,4.6,2.0,1.0,10,1100000.0,150000,sedan,4,F,...,"[big, big-trunk, wide-back-seats]",7ER,730L,2996,petrol,rwd,AT,258,7.8,9.8


Итого у нас остался 21 признак, включая целевой. Продолжим работу с пропусками, выбрасами и созданием новых признаков в ноутбуке на kaggle.

In [79]:
df.to_csv('autoru.csv')